# Projeto de Técnicas de Programação em Python - Transformação de Logs

<div align="center">
<img style="display: block; margin:auto; width: 50%;" src="assets/banco_log.png">
</div>

O projeto tem como premissa trabalhar com logs de servidor web em formato comum Apache. Para isso, foi consultada uma base de dados no kaggle por meio deste [link](https://www.kaggle.com/datasets/eliasdabbas/web-server-access-logs/data). 

A ideia é extrair os dados de interesse dos logs, transforma-los pelo Numpy e Pandas para depois tirar alguns insights.

# Indice


<div style="line-height: 2.0;">
    <div>1. Bibliotecas
    </div>
    <div>2. Corte do Arquivo
    </div>
    <div>3. Transformação dos Logs
        <div style="margin-left: 20px;">3.1. Extração dos caminhos para futura iteração</div>
        <div style="margin-left: 20px;">3.2. Aplicação da função</div>
    </div>
    <div>4. Conversão para o Numpy
        <div style="margin-left: 20px;">4.1. Criação do Numpy Vazio</div>
        <div style="margin-left: 20px;">4.2. Alimentação do Numpy</div>    
    </div>
    <div>5. Dataset com Logs
        <div style="margin-left: 20px;">5.1. Criação do Numpy Vazio</div>
        <div style="margin-left: 20px;">5.2. Alimentação do Numpy</div>    
        <div style="margin-left: 20px;">5.3. Verificação de Bots</div>            
    </div>
    <div>6. Insights
        <div style="margin-left: 20px;">6.1. Quantidade de acessos por Bots</div>
        <div style="margin-left: 20px;">6.2. Alimentação do Numpy</div>             
    </div>    
</div>

# 1. Bibliotecas

In [188]:
import numpy as np
import pandas as pd

# 2. Corte do arquivo

Como o arquivo original access.log possui 3.8GB, foi preciso selecionar uma quantidade de linhas para trabalhar mais facilmente. Por isso, fizemos a abertura do arquivo, lemos 1000 linhas e salvamos outro log.

**OBS**: Essa operação foi feita apenas para facilitar o processo. O arquivo original se encontra no site deste [link](https://www.kaggle.com/datasets/eliasdabbas/web-server-access-logs/data).

In [225]:
# with open("./../projeto/logs/access.log", "r") as file:
#     linhas_lidas = file.readlines()[:10001]

In [226]:
# with open("logs/access.log", "w") as file:
#     file.writelines(linhas_lidas)

# 3. Tranformação dos Logs

## 3.1 Compreensão da estrutura do Log

Primeiro, mostrarei como o log está estruturado.

In [227]:
with open("logs/access.log", "r") as file:
    teste = file.readline()
    print(teste)

54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"



Em resumo, essas são as informações de interesse no geral:

- IP do Cliente: Endereço IP do cliente solicitante
- Data: Data da solicitação 
- Hora: Hora da Solicitação
- Método HTTP: Qual método foi utilizado (GET, POST...)
- Pasta/Arquivo : Pasta e arquivo que foi acessado, ou se foi feito um Search
- Protocolo HTTP : Qual protocolo HTTP foi utilizado
- Código de Resposta: Indicação se a solicitação foi bem-sucedida ou não
- Tamanho da Resposta: Quantos bytes foram usados para aquisição da resposta
- User-Agent: Informações do navegador ou bot

## 3.2. Transformação dos logs

Primeiro, é preciso abrir o arquivo access.log:

In [266]:
with open("logs/access.log", "r") as file:
    arquivo = file.readlines()

Depois, é preciso extrair linha a linha do arquivo os valores das colunas:

In [273]:
# Definição do documento vazio que será preenchido com listas de itens a cada linha
documento = []

for linha in arquivo:

    # Remoção de todos os caracteres desnecessários para a leitura
    for e in ["[", "]", "(", ")", "{", "}", ";", '"', ","]:
        linha = linha.replace(e,"")

    # Separação por espaço ara selecionar como lista posteriormente
    i = linha.split(" ")
    
    ip = i[0]
    data = i[3][:11]
    hora = i[3][12:]
    metodo = i[5]

    # A pasta tem caracteristicas diferentes dependendo do log, por isso esse tratamento
    pasta = i[6].split("/")
    try:
        if pasta[1] == "m":
            pasta = pasta[2]
        else:
            pasta = pasta[1]
    except Exception:
            pasta = pasta[1]         
   
    if "?" in pasta:
        pasta = pasta.split("?")[0]


    http = i[7]
    resposta = i[8]
    tamanho_bytes = i[9]

    # Também há termos diferentes para o acesso, dependendo se for bot ou por onde está sendo o acesso
    if len(i) > 14:
        if i[12] == "compatible":
            user_agent = i[13]
        else:
            user_agent = f"{i[12]}-{i[13]}/{i[14]}"
    else:
        user_agent = {i[11]}

    # Transformação de todas as variáveis em um array para incluir no documento
    registro = [ip,data,hora,metodo,pasta,http,resposta,tamanho_bytes, user_agent]
    documento.append(registro) 

Cada log ficará assim no fim desse processo:

In [274]:
documento[2]

['31.56.96.51',
 '22/Jan/2019',
 '03:56:16',
 'GET',
 'image',
 'HTTP/1.1',
 '200',
 '5379',
 'Linux-Android/6.0']

# 4. Conversão para Numpy

## 4.1. Criação do Numpy Vazio

Para criação, foi utilizado o Numpy. O tipo mais utilizado foi Unicode, variando o tanto de caracteres necessários.

In [275]:
log_data_estruturado = np.empty(len(documento), dtype=[('id', 'U50'), ('data', 'U30'),('hora', 'U30'),('metodo', 'U20'), ('pasta', 'U50'),('http', 'U10'), ('resposta', 'U10'),('tamanho_bytes', int), ('user_agent', 'U100')])

## 4.2. Alimentação do Numpy 

In [276]:
for i, linha in enumerate(documento):
    ip,data,hora,metodo,pasta,http,resposta,tamanho_bytes, user_agent = linha
    log_data_estruturado[i] = (ip,data,hora,metodo,pasta,http,resposta,tamanho_bytes,user_agent)

In [277]:
log_data_estruturado[:10]

array([('54.36.149.41', '22/Jan/2019', '03:56:14', 'GET', 'filter', 'HTTP/1.1', '200', 30577, 'AhrefsBot/6.1'),
       ('31.56.96.51', '22/Jan/2019', '03:56:16', 'GET', 'image', 'HTTP/1.1', '200',  5667, 'Linux-Android/6.0'),
       ('31.56.96.51', '22/Jan/2019', '03:56:16', 'GET', 'image', 'HTTP/1.1', '200',  5379, 'Linux-Android/6.0'),
       ('40.77.167.129', '22/Jan/2019', '03:56:17', 'GET', 'image', 'HTTP/1.1', '200',  1696, 'bingbot/2.0'),
       ('91.99.72.15', '22/Jan/2019', '03:56:17', 'GET', 'product', 'HTTP/1.1', '200', 41483, 'Windows-NT/6.2'),
       ('40.77.167.129', '22/Jan/2019', '03:56:17', 'GET', 'image', 'HTTP/1.1', '200',  2654, 'bingbot/2.0'),
       ('40.77.167.129', '22/Jan/2019', '03:56:18', 'GET', 'image', 'HTTP/1.1', '200',  3688, 'bingbot/2.0'),
       ('40.77.167.129', '22/Jan/2019', '03:56:18', 'GET', 'image', 'HTTP/1.1', '200', 14776, 'bingbot/2.0'),
       ('66.249.66.194', '22/Jan/2019', '03:56:18', 'GET', 'filter', 'HTTP/1.1', '200', 34277, 'Googlebot/2

# 5. Dataset com os Logs

## 5.1. Importando o Dataframe

In [278]:
log_df = pd.DataFrame(data=log_data_estruturado)
log_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             10001 non-null  object
 1   data           10001 non-null  object
 2   hora           10001 non-null  object
 3   metodo         10001 non-null  object
 4   pasta          10001 non-null  object
 5   http           10001 non-null  object
 6   resposta       10001 non-null  object
 7   tamanho_bytes  10001 non-null  int32 
 8   user_agent     10001 non-null  object
dtypes: int32(1), object(8)
memory usage: 664.3+ KB


## 5.2. Conversão para Datetime

In [279]:
log_df["data"] = pd.to_datetime(log_df["data"], format="%d/%b/%Y")
log_df["hora"] = pd.to_datetime(log_df["hora"], format="%H:%M:%S").dt.time

In [280]:
log_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             10001 non-null  object        
 1   data           10001 non-null  datetime64[ns]
 2   hora           10001 non-null  object        
 3   metodo         10001 non-null  object        
 4   pasta          10001 non-null  object        
 5   http           10001 non-null  object        
 6   resposta       10001 non-null  object        
 7   tamanho_bytes  10001 non-null  int32         
 8   user_agent     10001 non-null  object        
dtypes: datetime64[ns](1), int32(1), object(7)
memory usage: 664.3+ KB


In [281]:
log_df

,id,data,hora,metodo,pasta,http,resposta,tamanho_bytes,user_agent
0,54.36.149.41,2019-01-22,03:56:14,GET,filter,HTTP/1.1,200,30577,AhrefsBot/6.1
1,31.56.96.51,2019-01-22,03:56:16,GET,image,HTTP/1.1,200,5667,Linux-Android/6.0
2,31.56.96.51,2019-01-22,03:56:16,GET,image,HTTP/1.1,200,5379,Linux-Android/6.0
3,40.77.167.129,2019-01-22,03:56:17,GET,image,HTTP/1.1,200,1696,bingbot/2.0
4,91.99.72.15,2019-01-22,03:56:17,GET,product,HTTP/1.1,200,41483,Windows-NT/6.2
...,...,...,...,...,...,...,...,...,...
9996,192.15.6.66,2019-01-22,04:36:57,GET,product,HTTP/1.1,302,0,Linux-Android/8.0.0
9997,37.129.232.66,2019-01-22,04:36:57,GET,static,HTTP/1.1,200,5807,Linux-Android/7.0
9998,37.129.232.66,2019-01-22,04:36:57,GET,static,HTTP/1.1,200,7356,Linux-Android/7.0
9999,37.129.232.66,2019-01-22,04:36:57,GET,static,HTTP/1.1,200,6496,Linux-Android/7.0


## 5.3. Verificação de Bots

Para facilitar a pesquisa de bots, criaremos uma coluna "bot?" e atribuir True ou False. Para isso, consideraremos se há a palavra bot no user_agent:

In [282]:
log_df["eh_bot"] = log_df["user_agent"] .apply(lambda x: True if "bot" in x else False)

In [283]:
log_df

,id,data,hora,metodo,pasta,http,resposta,tamanho_bytes,user_agent,eh_bot
0,54.36.149.41,2019-01-22,03:56:14,GET,filter,HTTP/1.1,200,30577,AhrefsBot/6.1,False
1,31.56.96.51,2019-01-22,03:56:16,GET,image,HTTP/1.1,200,5667,Linux-Android/6.0,False
2,31.56.96.51,2019-01-22,03:56:16,GET,image,HTTP/1.1,200,5379,Linux-Android/6.0,False
3,40.77.167.129,2019-01-22,03:56:17,GET,image,HTTP/1.1,200,1696,bingbot/2.0,True
4,91.99.72.15,2019-01-22,03:56:17,GET,product,HTTP/1.1,200,41483,Windows-NT/6.2,False
...,...,...,...,...,...,...,...,...,...,...
9996,192.15.6.66,2019-01-22,04:36:57,GET,product,HTTP/1.1,302,0,Linux-Android/8.0.0,False
9997,37.129.232.66,2019-01-22,04:36:57,GET,static,HTTP/1.1,200,5807,Linux-Android/7.0,False
9998,37.129.232.66,2019-01-22,04:36:57,GET,static,HTTP/1.1,200,7356,Linux-Android/7.0,False
9999,37.129.232.66,2019-01-22,04:36:57,GET,static,HTTP/1.1,200,6496,Linux-Android/7.0,False


# 6. Insights

## 6.1. Quantidade de acessos por Bots

In [284]:
verificacao_bot_df = log_df.groupby("eh_bot").size()
verificacao_bot_df[True]

2470

In [285]:
porcentagem_bots = verificacao_bot_df[True]/log_df["eh_bot"].count()

print(f"A quantidade de logs referentes a bots é de {verificacao_bot_df[True]}, {round(porcentagem_bots*100,2)}% do total de acessos")

A quantidade de logs referentes a bots é de 2470, 24.7% do total de acessos


## 6.2. Arquivos Mais Acessados

In [286]:
arquivos_mais_acessados = log_df.groupby("pasta").size().sort_values(ascending=False)
arquivos_mais_acessados.head()

pasta
image      3378
filter     3331
product    1002
static      882
browse      249
dtype: int64